In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [7]:
data = pd.read_csv("prj-ML-model-LT_OV30/modeling_data.csv", encoding = "cp949")
print(data.head())
print(data.shape)

            dlvry_id  reg_hour  ord_price  actual_dlvry_distance  pick_floor  \
0  20230827_9376QTJN        11      22500                   1274         1.0   
1  20230827_1765MBEB        17      15700                   2809         1.0   
2  20230827_0741UTNW        18      20400                   1961        -1.0   
3  20230827_6306VAVZ        19      16900                   1826         1.0   
4  20230827_5544DEBH        11      21000                   1184         2.0   

  pick_rgn2_nm pick_rgn3_nm pick_category pick_건물용도  dlvry_지상층수  dlvry_지하층수  \
0          서초구          잠원동       CHINESE      업무시설         NaN         NaN   
1          서초구          반포동         SNACK     일반음식점        19.0         5.0   
2          서초구          서초동     ZZIM_TANG       아파트         4.0         1.0   
3          서초구          방배동         ASIAN       아파트         5.0         0.0   
4          서초구          서초동          CAFE       NaN         4.0         1.0   

  dlvry_건물용도 day_of_week  is_holiday    기온  

In [8]:
# 건물 정보 없는 데이터 삭제 
data = data.dropna(subset=['pick_건물용도', 'dlvry_건물용도'])
print(data.shape) #44644 

(44644, 19)


## 1. 데이터 전처리
###### pick_floor : 지하, 중층, 상층
###### pick_건물용도/dlvry_건물용도 : 개수 줄이기

## 1-1. pick_건물용도/ dlvry_건물용도 : 개수 축소

In [6]:
print(data['pick_건물용도'].value_counts())
# 업무시설 19609
# 

pick_건물용도
업무시설          19609
근린생활시설        10639
아파트            7272
일반음식점          6911
일반상점           5243
공동주택           4224
교육시설           2148
의료시설           1547
백화점             726
자동차관련시설         608
오피스텔            380
기타시설            279
숙박시설            235
대형판매점           212
위락시설            167
문화시설            113
종교시설             41
체육시설             21
위험물저장및처리시설       16
운수시설              2
Name: count, dtype: int64


### 1-2. dlvry_지상층수 /지하층수 : dlvry_건물용도 median 값 가져오기

In [ ]:
median_values = data.groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].median()
mean_values = data.groupby('dlvry_건물용도')[['dlvry_지상층수', 'dlvry_지하층수']].mean()

print("Median Values:")
print(median_values)

print("\nMean Values:")
print(mean_values)
